In [6]:
import pathlib
from collections import defaultdict
import numpy as np
import pandas as pd
from obspy import read, Stream
import hvsrpy
from hvsrpy.settings import HvsrPreProcessingSettings, HvsrTraditionalProcessingSettings

# ✅ 데이터 폴더 지정
data_dir = pathlib.Path("C:/SOLODATA/zonghap/3month/3월부터얻은데이터목록")
sac_files = sorted(data_dir.glob("*.sac"))

# ✅ Z/N/E 그룹화
grouped_files = defaultdict(dict)
for file in sac_files:
    parts = file.stem.split(".")
    if len(parts) < 9:
        continue
    base_id = ".".join(parts[:-1])
    comp = parts[-1]
    grouped_files[base_id][comp] = str(file)

fname_sets = []
for base, comps in grouped_files.items():
    if all(k in comps for k in ["E", "N", "Z"]):
        fname_sets.append((base, [comps["Z"], comps["N"], comps["E"]]))

# ✅ 저장 폴더
save_root = pathlib.Path("C:/SOLODATA/zonghap/hvsr_try")
output_dir = save_root / data_dir.name
output_dir.mkdir(parents=True, exist_ok=True)

# ✅ 전처리 설정
preproc = HvsrPreProcessingSettings()
preproc.window_length_in_seconds = 100
preproc.detrend = "linear"

# ✅ 처리 설정
proc = HvsrTraditionalProcessingSettings()
proc.method_to_combine_horizontals = "geometric_mean"
proc.smoothing = dict(operator="konno_and_ohmachi", bandwidth=40,
                      center_frequencies_in_hz=np.geomspace(0.2, 50, 200))

# ✅ 세트별 반복 처리
for base_id, (z_file, n_file, e_file) in fname_sets:
    try:
        sac_list = [z_file, n_file, e_file]  # 반드시 리스트로 구성
        tr_z = read(z_file)[0]
        endtime = tr_z.stats.endtime

        results = []
        for n in range(300, 3601, 300):
            start = endtime - n

            # 1. 파일에서 읽기
            srecords = hvsrpy.read(sac_list)

            # 2. 전처리
            srecords = hvsrpy.preprocess(srecords, preproc)

            # 3. 각 trace에 대해 trim 적용
            for rec in srecords:
                rec.trim(starttime=start)

            # 4. 처리
            hvsr_result = hvsrpy.process(srecords, proc)

            # 5. f0 추출
            f0 = hvsr_result["f0"]
            if f0 is None or np.isnan(f0):
                f0 = "NaN"

            results.append((n, f0))

        # 6. CSV 저장
        df = pd.DataFrame(results, columns=["time(s)", "resonant frequency"])
        csv_path = output_dir / f"{base_id}_f0_values.csv"
        df.to_csv(csv_path, index=False)
        print(f"✅ 저장 완료: {csv_path}")

    except Exception as e:
        print(f"⚠️ 오류 발생: {base_id} - {e}")


⚠️ 오류 발생: 453015908.0001.2025.03.23.03.34.18.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0002.2025.03.23.04.40.34.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0003.2025.03.23.05.43.22.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0004.2025.03.23.07.20.34.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0005.2025.03.23.08.25.18.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0006.2025.03.25.06.28.50.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0007.2025.03.25.07.35.10.000 - Must provide 3 peer files (one per trace) as list or tuple, not <class 'str'>.
⚠️ 오류 발생: 453015908.0008.2025.04.01.04.52.50.000 - Must provide 3 peer files (one per trac